CARGA DE DADOS ORIGINAIS 
informando a carga de dados originais e separando entre características e classe alvo

In [30]:
import pandas as pd 
import numpy as np
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [31]:
#definido o título da colunas 
headers = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide',
            'total sulfur dioxide','density','pH','sulphates','alcohol','quality']

#carga do conjunto de treino
file = 'winequality-red.csv'
data_train = pd.read_csv(file, sep=';', header=None, names=headers)
#data_train.info(verbose=True)

#Transfomaçao dos atributos e da classe alvo em matriz
X_train_ = np.array(data_train.iloc[:, 0:11])
y_train_ = np.array(data_train['quality'])

#Separação de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_, train_size = 0.2, random_state=31)

#Carga de conjunto de teste
file = 'winequality-red.csv'
data_test = pd.read_csv(file, sep=';', header=None, names=headers)

#Transformação dos atributos e da classe alvo em matriz
X_test = np.array(data_test.iloc[:,0:11])
y_test = np.array(data_train['quality'])

In [32]:
#checando de parte dos dados carregados
print(data_train.head(), 
        data_test.head(), 
        X_train[0], 
        y_train[0], 
        sep='\n\n')

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0  fixed acidity  volatile acidity  citric acid  residual sugar  chlorides   
1            7.4               0.7            0             1.9      0.076   
2            7.8              0.88            0             2.6      0.098   
3            7.8              0.76         0.04             2.3      0.092   
4           11.2              0.28         0.56             1.9      0.075   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0  free sulfur dioxide  total sulfur dioxide  density    pH  sulphates   
1                   11                    34   0.9978  3.51       0.56   
2                   25                    67   0.9968   3.2       0.68   
3                   15                    54    0.997  3.26       0.65   
4                   17                    60    0.998  3.16       0.58   

   alcohol  quality  
0  alcohol  quality  
1      9.4        5  
2      9.8        5 

In [33]:
data_train.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600
unique,97,144,81,92,154,61,145,437,90,97,66,7
top,7.2,0.6,0,2,0.08,6,28,0.9972,3.3,0.6,9.5,5
freq,67,47,132,156,66,138,43,36,57,69,139,681


NORMALIZAÇÃO DOS DADOS 
Como pode ser vistos nas saídas do final da seção as características estão em ordens de grandeza diferentes. Para muitos algoritmos este é um problema que pode levar a modelos inadequados. Para resolver este problema realizaremos a normalização dos dados.

In [40]:
#Criação do transformador para normalização
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()


In [41]:
#Normalização das características
X_train = norm.fit_transform(X_train)
X_test = norm.fit_transform(X_test)
X_val = norm.fit_transform(X_val)

ValueError: could not convert string to float: 'fixed acidity'

In [42]:
#checando a parte de dados normalizados 
print(X_train[0],
        X_val[0],
        X_test[0],
        sep="\n\n")

[-0.36832844  0.70376003 -0.32100428 -0.17817278 -0.26931748  1.43977149
  2.65375455 -0.60760869 -0.22487088 -1.1488296  -0.42572627]

['7.8' '0.53' '0.01' '1.6' '0.077' '3' '19' '0.995' '3.16' '0.46' '9.8']

['fixed acidity' 'volatile acidity' 'citric acid' 'residual sugar'
 'chlorides' 'free sulfur dioxide' 'total sulfur dioxide' 'density' 'pH'
 'sulphates' 'alcohol']


VAÁRIAVEIS "DUMMIES" PARA TRATAMENTO DE DADOS CATEGORICOS NOMINAIS
Algumas variáveis no conjunto de dados são discretas nominais, ou seja, os valores apresentados não representam uma contagem ordinal, mas categorias. Essa situação também pode gerar inconsistências na criação e treinamento de modelos. Para lidar com essa situação serão criadas variáveis dummies, ou seja, variáveis que representam cada categoria e assumem valores binárias (1 = verdadeiro / 0 = falso).

In [ ]:
data_train.head(15)

In [ ]:
#conferindo os valores da variavel fixed acidity
print(data_train['fixed acidity'].head(10),
        data_train['fixed acidity'].describe(),
        sep='\n\n')

In [ ]:
#Coferindo os valores possíveis da variavel
set(data_train['fixed acidity'])

Percebemos pala analise que as opções de valores para fixed acidity estão registradas na base de dados com os valores []. Para criação de variaveis *dummines* serão criadas $n-1$ variaveis binárias, onde $n$ é o número de categorias. Isso é feito porque a categoria $n$ ocorrerá quando todas as demais tiveram valor 0 e, portanto, não precise ser mapeada.

Para exemplificar a lógica consideremos fixed acidity cuja categoria é 0. podemos consultar o dataframe original da forma abaixo e obtemos um vetor de valores lógicos, na qual é apontado verdadeiro para os casos emq ue a categoria do fixed acidity é 0.

In [ ]:
data_train['fixed acidity'] == 0

In [ ]:
fixed_acidity_0 = (data_train['fixed acidity'] ==0)
fixed_acidity_0

Para criar as variaveis, basta usar esta mesma lógica para as categorias desejadas, transformando os valores lógicos em inteiros. Contudo, a biblioteca pandas possui um método get_dummies que simplemente transforma. Aplicaremos abaixo o método no conjuto de dados

In [ ]:
pd.get_dummies(data = data_train,
                prefix='fa',
                columns=['fixed acidity'],
                drop_first=False)

Veja que o atributo 'fixed acidity' foi substituído pelas variáveis novas e criado um data frame com os dados originais, exceto por esta substituição. Além disso, veja que foi definido o parâmetro drop_first com valor True, isso força o método a eliminar a primeira categoria, tornando a quantidade de dummies igual a $n-1$ pelos motivos explicados anteriormente nessa seção. Caso queira que todas as categorias estejam presentes no novo data frame, o parâmetro drop_first deve ser descartado, tendo em vista que seu valor padrão é False.

In [ ]:
#usando a lógica estado para recarregar os dados e refazer a separação
data_train_ = pd.get_dummies(data = data_train,
                prefix='fa',
                columns=['fixed acidity'],
                drop_first=False)
data_train_.head()

In [ ]:
data_test_ = pd.get_dummies(data = data_test,
                prefix='fa',
                columns=['fixed acidity'],
                drop_first=False)
data_test_.head()

In [ ]:
#Separação da variavel algo não mudo 
#-----------------------------------

#Transoformação dos atributos e da classe alvo em matrizes 
y_train_ = np.array(data_train['quality'])

#Transformação dos atributos e da classe alvo em matriz
y_test = np.array(data_test['quality'])

#varificando 
print(y_train_[:10],
        y_test[:10],
        sep='\n\n')

In [ ]:
#Separando os atributos precisa atentar que não preservada a ordem das colunas
#-----------------------------------------------------------------------------

#Recuperação dos nomes das colunas 
atributos = list(data_train_.columns)

#verificando 
atributos

In [ ]:
#'eliminidando as colunas com a classe na lista de atributos'
atributos.remove('quality')

#verificando 
atributos

In [ ]:
#conferindo o uso da lista de atributos para separação do conjunto de dados 
data_train_.loc[:, atributos]

In [ ]:

#Uso da lista de atributos para separação do conjunto de dados

#Transoformação dos atributos e da classe alvo em matrizes
X_train_ = np.array(data_train_.loc[:, atributos])

#Transformação dos atributos e da classe alvo em matrizes
X_test = np.array(data_test_.loc[:, atributos])

In [ ]:
#Separação de treino e validação não muda
##=======================================

#Separação de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_,
                                                  train_size=0.8,
                                                  random_state=31)

In [ ]:

#Checagem rápida de parte dos dados carregados
print(X_train[:5],
      X_val[:5],
      X_test[:5],
      sep='\n\n')